In [26]:
import cv2
import os
import numpy as np

print(os.getcwd())

def make_df(train_path, test_path, img_size):
    train_ids = next(os.walk(train_path))[1]
    test_ids = next(os.walk(test_path))[1]
    X_train = np.zeros((len(train_ids), img_size, img_size, 3), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), img_size, img_size, 1), dtype=np.bool)
    for i, id_ in enumerate(train_ids):
        path = train_path+id_
        img = cv2.imread(path+'/images/'+id_+'.png')
        img = cv2.resize(img, (img_size, img_size))
        X_train[i]=img
        mask=np.zeros((img_size, img_size, 1), dtype=np.bool)
        for mask_file in next(os.walk(path+'/masks/'))[2]:
            mask_=cv2.imread(path+'/masks/'+mask_file,0)
            mask_=cv2.resize(mask_, (img_size, img_size))
            mask_=mask_[:,:,np.newaxis]
            mask=np.maximum(mask, mask_)
        Y_train[i]=mask
    X_test=np.zeros((len(test_ids), img_size, img_size, 3), dtype=np.uint8)
    sizes_test=[]
    for i, id_ in enumerate(test_ids):
        path=test_path+id_
        img=cv2.imread(path+'/images/'+id_+'.png')
        sizes_test.append([img.shape[0], img.shape[1]])
        img=cv2.resize(img, (img_size, img_size))
        X_test[i]=img
    return X_train, Y_train, X_test, sizes_test    

C:\Public\DataScienceBowl


## Define the U-net model 

In [27]:
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda


def Unet_model(img_size):
    inputs = Input((img_size, img_size, 3))
    s=Lambda(lambda x:x/255)(inputs) #Normalization
    
    #Downsampling [Increase the depth but decrease the height]
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.summary()
    
    return model

## Define the Generator. Using Keras ImageDataGenerator. 

In [45]:
from keras.preprocessing.image import ImageDataGenerator

def generator(xtr, xval, ytr, yval, batch_size):
#     data_gen_args = dict(horizontal_flip=True,
#                          vertical_flip=True,
#                          rotation_range=90.,
#                          width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.1)
    data_gen_args = dict(shear_range= 0.5,
                         rotation_range = 50.,
                         zoom_range=0.2,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         fill_mode='reflect')
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    image_datagen.fit(xtr, seed=7)
    mask_datagen.fit(ytr, seed=7)
    image_generator = image_datagen.flow(xtr, batch_size=batch_size, seed=7)
    mask_generator = mask_datagen.flow(ytr, batch_size=batch_size, seed=7)
    train_generator = zip(image_generator, mask_generator)

    val_gen_args = dict()
    image_datagen_val = ImageDataGenerator(**val_gen_args)
    mask_datagen_val = ImageDataGenerator(**val_gen_args)
    image_datagen_val.fit(xval, seed=7)
    mask_datagen_val.fit(yval, seed=7)
    image_generator_val = image_datagen_val.flow(xval, batch_size=batch_size, seed=7)
    mask_generator_val = mask_datagen_val.flow(yval, batch_size=batch_size, seed=7)
    val_generator = zip(image_generator_val, mask_generator_val)

    return train_generator, val_generator

In [46]:
import tensorflow as tf
import numpy as np
from keras import backend as K

def mean_iou(y_true, y_pred):
    prec=[]
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_=tf.to_int32(y_pred>t)
        score, up_opt=tf.metrics.mean_iou(y_true, y_pred_,2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score=tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec))

In [47]:
from skimage.morphology import label

# Given rle implementations
def rle_encoding(x):
    dots=np.where(x.T.flatten()==1)[0]
    run_lengths=[]
    prev=-2
    for b in dots:
        if (b>prev+1):run_lengths.extend((b+1,0))
        run_lengths[-1]+=1
        prev=b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img=label(x>cutoff)
    for i in range(1, lab_img.max()+1):
        yield rle_encoding(lab_img==i)

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import os
import numpy as np
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import skimage.transform
import skimage.morphology

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

#For some visualiations
import sklearn.model_selection 
import keras.preprocessing.image # For image generation and augmentation
import datetime
import cv2
import matplotlib.cm as cm # color map

import tensorflow as tf
%matplotlib inline

if __name__=="__main__":
    img_size=256
    batch_size=32
    path = os.getcwd()
    train_path = path+'\\stage1_train\\'
    test_path = path+'\\stage1_test\\'
    
    X_train, Y_train, X_test, sizes_test = make_df(train_path, test_path, img_size)
    xtr, xval, ytr, yval = train_test_split(X_train, Y_train, test_size=0.1, random_state=7)
    train_generator, val_generator = generator(xtr, xval, ytr, yval, batch_size)
    
    model=Unet_model(img_size)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer=ModelCheckpoint('unet-modeldsbowl-dataaug-ep5.hd5', verbose=1, save_best_only=True)
    model.fit_generator(train_generator, steps_per_epoch=250, epochs=5, validation_data=val_generator, validation_steps=len(xval)/batch_size, callbacks=[earlystopper, checkpointer])
    #model.fit_generator(train_generator, steps_per_epoch=len(xtr)/6, epochs=50, validation_data=val_generator, validation_steps=len(xval)/batch_size)
    preds_test=model.predict(X_test, verbose=1)
    
    preds_test_unsampled=[]
    for i in range(len(preds_test)):
        preds_test_unsampled.append(cv2.resize(preds_test[i], (sizes_test[i][1], sizes_test[i][0])))
        test_ids=next(os.walk(test_path))[1]
        new_test_ids=[]
        rles=[]

    for n, id_ in enumerate(test_ids):
        #print(n, id_)
        rle = list(prob_to_rles(preds_test_unsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_]*len(rle))
    sub=pd.DataFrame()
    sub['ImageId']=new_test_ids
    sub['EncodedPixels']=pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    sub.to_csv('subDataScienceBowl2018_unet_dataaug_v1.csv', index=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 256, 256, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_117 (Conv2D)             (None, 256, 256, 16) 448         lambda_8[0][0]                   
__________________________________________________________________________________________________
dropout_56 (Dropout)            (None, 256, 256, 16) 0           conv2d_117[0][0]                 
__________________________________________________________________________________________________
conv2d_118

__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 256, 256, 16) 4624        concatenate_28[0][0]             
__________________________________________________________________________________________________
dropout_64 (Dropout)            (None, 256, 256, 16) 0           conv2d_133[0][0]                 
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 256, 256, 16) 2320        dropout_64[0][0]                 
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 256, 256, 1)  17          conv2d_134[0][0]                 
Total params: 1,941,105
Trainable params: 1,941,105
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/5


177/250 [====================>.........] - ETA: 1:21:26 - loss: 0.7169 - mean_iou: 0.0000e+0 - ETA: 1:13:16 - loss: 0.6441 - mean_iou: 0.2110    - ETA: 1:10:07 - loss: 0.6106 - mean_iou: 0.287 - ETA: 1:08:31 - loss: 0.5620 - mean_iou: 0.324 - ETA: 1:07:22 - loss: 0.5328 - mean_iou: 0.347 - ETA: 1:06:29 - loss: 0.5229 - mean_iou: 0.362 - ETA: 1:05:48 - loss: 0.5121 - mean_iou: 0.374 - ETA: 1:05:17 - loss: 0.4977 - mean_iou: 0.382 - ETA: 1:04:49 - loss: 0.4898 - mean_iou: 0.388 - ETA: 1:04:23 - loss: 0.4789 - mean_iou: 0.393 - ETA: 1:03:59 - loss: 0.5757 - mean_iou: 0.398 - ETA: 1:03:43 - loss: 0.5666 - mean_iou: 0.401 - ETA: 1:03:21 - loss: 0.5610 - mean_iou: 0.404 - ETA: 1:03:00 - loss: 0.5498 - mean_iou: 0.407 - ETA: 1:02:45 - loss: 0.5420 - mean_iou: 0.409 - ETA: 1:02:22 - loss: 0.5372 - mean_iou: 0.411 - ETA: 1:02:02 - loss: 0.5330 - mean_iou: 0.413 - ETA: 1:01:41 - loss: 0.5267 - mean_iou: 0.414 - ETA: 1:00:52 - loss: 0.5228 - mean_iou: 0.415 - ETA: 1:00:34 - loss: 0.5159 - mean_io

250/250 [==============================] - ETA: 18:45 - loss: 0.2337 - mean_iou: 0.522 - ETA: 18:29 - loss: 0.2330 - mean_iou: 0.523 - ETA: 18:14 - loss: 0.2323 - mean_iou: 0.524 - ETA: 17:58 - loss: 0.2318 - mean_iou: 0.524 - ETA: 17:42 - loss: 0.2312 - mean_iou: 0.525 - ETA: 17:27 - loss: 0.2304 - mean_iou: 0.525 - ETA: 17:11 - loss: 0.2298 - mean_iou: 0.526 - ETA: 16:56 - loss: 0.2292 - mean_iou: 0.527 - ETA: 16:40 - loss: 0.2288 - mean_iou: 0.527 - ETA: 16:24 - loss: 0.2281 - mean_iou: 0.528 - ETA: 16:09 - loss: 0.2274 - mean_iou: 0.529 - ETA: 15:53 - loss: 0.2268 - mean_iou: 0.529 - ETA: 15:37 - loss: 0.2263 - mean_iou: 0.530 - ETA: 15:21 - loss: 0.2256 - mean_iou: 0.530 - ETA: 15:06 - loss: 0.2252 - mean_iou: 0.531 - ETA: 14:50 - loss: 0.2246 - mean_iou: 0.532 - ETA: 14:34 - loss: 0.2242 - mean_iou: 0.532 - ETA: 14:19 - loss: 0.2234 - mean_iou: 0.533 - ETA: 14:03 - loss: 0.2228 - mean_iou: 0.534 - ETA: 13:48 - loss: 0.2222 - mean_iou: 0.534 - ETA: 13:32 - loss: 0.2217 - mean_iou:

177/250 [====================>.........] - ETA: 1:05:03 - loss: 0.0968 - mean_iou: 0.689 - ETA: 1:04:59 - loss: 0.0948 - mean_iou: 0.690 - ETA: 1:04:33 - loss: 0.0938 - mean_iou: 0.690 - ETA: 1:04:13 - loss: 0.0950 - mean_iou: 0.690 - ETA: 1:04:09 - loss: 0.0972 - mean_iou: 0.690 - ETA: 1:03:53 - loss: 0.0941 - mean_iou: 0.690 - ETA: 1:03:34 - loss: 0.0935 - mean_iou: 0.691 - ETA: 1:03:19 - loss: 0.0952 - mean_iou: 0.691 - ETA: 1:03:02 - loss: 0.0970 - mean_iou: 0.691 - ETA: 1:02:44 - loss: 0.0963 - mean_iou: 0.691 - ETA: 1:02:26 - loss: 0.0982 - mean_iou: 0.692 - ETA: 1:02:10 - loss: 0.0983 - mean_iou: 0.692 - ETA: 1:01:54 - loss: 0.0984 - mean_iou: 0.692 - ETA: 1:01:40 - loss: 0.0973 - mean_iou: 0.693 - ETA: 1:01:25 - loss: 0.0977 - mean_iou: 0.693 - ETA: 1:00:33 - loss: 0.0980 - mean_iou: 0.693 - ETA: 1:00:18 - loss: 0.0998 - mean_iou: 0.693 - ETA: 1:00:05 - loss: 0.0988 - mean_iou: 0.694 - ETA: 59:51 - loss: 0.0989 - mean_iou: 0.6943  - ETA: 59:37 - loss: 0.0984 - mean_iou: 0.694 -

...

250/250 [==============================] - ETA: 18:41 - loss: 0.0794 - mean_iou: 0.779 - ETA: 18:26 - loss: 0.0795 - mean_iou: 0.779 - ETA: 18:10 - loss: 0.0794 - mean_iou: 0.779 - ETA: 17:55 - loss: 0.0794 - mean_iou: 0.779 - ETA: 17:39 - loss: 0.0794 - mean_iou: 0.779 - ETA: 17:23 - loss: 0.0794 - mean_iou: 0.779 - ETA: 17:07 - loss: 0.0794 - mean_iou: 0.779 - ETA: 16:52 - loss: 0.0794 - mean_iou: 0.779 - ETA: 16:36 - loss: 0.0794 - mean_iou: 0.779 - ETA: 16:20 - loss: 0.0793 - mean_iou: 0.779 - ETA: 16:05 - loss: 0.0794 - mean_iou: 0.779 - ETA: 15:49 - loss: 0.0794 - mean_iou: 0.779 - ETA: 15:34 - loss: 0.0794 - mean_iou: 0.779 - ETA: 15:18 - loss: 0.0794 - mean_iou: 0.780 - ETA: 15:03 - loss: 0.0794 - mean_iou: 0.780 - ETA: 14:47 - loss: 0.0793 - mean_iou: 0.780 - ETA: 14:32 - loss: 0.0794 - mean_iou: 0.780 - ETA: 14:16 - loss: 0.0794 - mean_iou: 0.780 - ETA: 14:00 - loss: 0.0793 - mean_iou: 0.780 - ETA: 13:45 - loss: 0.0793 - mean_iou: 0.780 - ETA: 13:29 - loss: 0.0793 - mean_iou:

177/250 [====================>.........] - ETA: 1:04:52 - loss: 0.0583 - mean_iou: 0.797 - ETA: 1:05:34 - loss: 0.0715 - mean_iou: 0.797 - ETA: 1:08:37 - loss: 0.0696 - mean_iou: 0.797 - ETA: 1:08:57 - loss: 0.0698 - mean_iou: 0.797 - ETA: 1:07:45 - loss: 0.0712 - mean_iou: 0.797 - ETA: 1:06:50 - loss: 0.0730 - mean_iou: 0.797 - ETA: 1:06:06 - loss: 0.0723 - mean_iou: 0.797 - ETA: 1:05:31 - loss: 0.0733 - mean_iou: 0.797 - ETA: 1:04:59 - loss: 0.0747 - mean_iou: 0.797 - ETA: 1:03:33 - loss: 0.0763 - mean_iou: 0.797 - ETA: 1:03:11 - loss: 0.0740 - mean_iou: 0.797 - ETA: 1:02:51 - loss: 0.0750 - mean_iou: 0.797 - ETA: 1:02:31 - loss: 0.0748 - mean_iou: 0.797 - ETA: 1:02:13 - loss: 0.0755 - mean_iou: 0.797 - ETA: 1:01:53 - loss: 0.0745 - mean_iou: 0.797 - ETA: 1:01:33 - loss: 0.0745 - mean_iou: 0.797 - ETA: 1:01:15 - loss: 0.0753 - mean_iou: 0.797 - ETA: 1:00:57 - loss: 0.0754 - mean_iou: 0.797 - ETA: 1:00:39 - loss: 0.0752 - mean_iou: 0.797 - ETA: 1:00:22 - loss: 0.0761 - mean_iou: 0.798

250/250 [==============================] - ETA: 18:41 - loss: 0.0743 - mean_iou: 0.804 - ETA: 18:25 - loss: 0.0743 - mean_iou: 0.804 - ETA: 18:10 - loss: 0.0743 - mean_iou: 0.804 - ETA: 17:53 - loss: 0.0744 - mean_iou: 0.804 - ETA: 17:38 - loss: 0.0743 - mean_iou: 0.804 - ETA: 17:22 - loss: 0.0745 - mean_iou: 0.804 - ETA: 17:07 - loss: 0.0746 - mean_iou: 0.804 - ETA: 16:51 - loss: 0.0747 - mean_iou: 0.804 - ETA: 16:36 - loss: 0.0747 - mean_iou: 0.804 - ETA: 16:20 - loss: 0.0746 - mean_iou: 0.804 - ETA: 16:05 - loss: 0.0745 - mean_iou: 0.804 - ETA: 15:49 - loss: 0.0744 - mean_iou: 0.804 - ETA: 15:34 - loss: 0.0745 - mean_iou: 0.804 - ETA: 15:18 - loss: 0.0745 - mean_iou: 0.804 - ETA: 15:03 - loss: 0.0745 - mean_iou: 0.804 - ETA: 14:47 - loss: 0.0745 - mean_iou: 0.804 - ETA: 14:31 - loss: 0.0745 - mean_iou: 0.804 - ETA: 14:16 - loss: 0.0744 - mean_iou: 0.804 - ETA: 14:00 - loss: 0.0744 - mean_iou: 0.804 - ETA: 13:45 - loss: 0.0744 - mean_iou: 0.804 - ETA: 13:29 - loss: 0.0744 - mean_iou:

177/250 [====================>.........] - ETA: 1:04:28 - loss: 0.0812 - mean_iou: 0.814 - ETA: 1:04:26 - loss: 0.0774 - mean_iou: 0.814 - ETA: 1:04:19 - loss: 0.0726 - mean_iou: 0.814 - ETA: 1:04:11 - loss: 0.0710 - mean_iou: 0.814 - ETA: 1:03:52 - loss: 0.0742 - mean_iou: 0.814 - ETA: 1:03:37 - loss: 0.0746 - mean_iou: 0.814 - ETA: 1:01:56 - loss: 0.0760 - mean_iou: 0.814 - ETA: 1:01:51 - loss: 0.0750 - mean_iou: 0.814 - ETA: 1:01:46 - loss: 0.0748 - mean_iou: 0.814 - ETA: 1:01:37 - loss: 0.0753 - mean_iou: 0.814 - ETA: 1:01:27 - loss: 0.0741 - mean_iou: 0.814 - ETA: 1:01:16 - loss: 0.0744 - mean_iou: 0.814 - ETA: 1:01:09 - loss: 0.0747 - mean_iou: 0.814 - ETA: 1:00:54 - loss: 0.0741 - mean_iou: 0.814 - ETA: 1:00:42 - loss: 0.0736 - mean_iou: 0.814 - ETA: 1:00:28 - loss: 0.0726 - mean_iou: 0.814 - ETA: 1:00:16 - loss: 0.0721 - mean_iou: 0.814 - ETA: 1:00:02 - loss: 0.0729 - mean_iou: 0.814 - ETA: 59:48 - loss: 0.0737 - mean_iou: 0.8146  - ETA: 59:36 - loss: 0.0726 - mean_iou: 0.814 -

250/250 [==============================] - ETA: 18:39 - loss: 0.0727 - mean_iou: 0.818 - ETA: 18:24 - loss: 0.0727 - mean_iou: 0.818 - ETA: 18:08 - loss: 0.0728 - mean_iou: 0.818 - ETA: 17:53 - loss: 0.0728 - mean_iou: 0.818 - ETA: 17:37 - loss: 0.0727 - mean_iou: 0.818 - ETA: 17:22 - loss: 0.0727 - mean_iou: 0.818 - ETA: 17:06 - loss: 0.0727 - mean_iou: 0.818 - ETA: 16:51 - loss: 0.0728 - mean_iou: 0.818 - ETA: 16:35 - loss: 0.0727 - mean_iou: 0.818 - ETA: 16:20 - loss: 0.0726 - mean_iou: 0.818 - ETA: 16:04 - loss: 0.0726 - mean_iou: 0.818 - ETA: 15:49 - loss: 0.0726 - mean_iou: 0.818 - ETA: 15:33 - loss: 0.0725 - mean_iou: 0.818 - ETA: 15:18 - loss: 0.0725 - mean_iou: 0.818 - ETA: 15:02 - loss: 0.0726 - mean_iou: 0.818 - ETA: 14:47 - loss: 0.0726 - mean_iou: 0.818 - ETA: 14:31 - loss: 0.0725 - mean_iou: 0.818 - ETA: 14:16 - loss: 0.0724 - mean_iou: 0.818 - ETA: 14:00 - loss: 0.0725 - mean_iou: 0.818 - ETA: 13:44 - loss: 0.0725 - mean_iou: 0.818 - ETA: 13:28 - loss: 0.0724 - mean_iou:

In [39]:
    preds_test_unsampled=[]
    for i in range(len(preds_test)):
        preds_test_unsampled.append(cv2.resize(preds_test[i], (sizes_test[i][1], sizes_test[i][0])))
        test_ids=next(os.walk(test_path))[1]
        new_test_ids=[]
        rles=[]

    for n, id_ in enumerate(test_ids):
        #print(n, id_)
        rle = list(prob_to_rles(preds_test_unsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_]*len(rle))
    sub=pd.DataFrame()
    sub['ImageId']=new_test_ids
    sub['EncodedPixels']=pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    sub.to_csv('subDataScienceBowl2018_unet_dataaug_v1.csv')

In [40]:
for i in range(len(preds_test_unsampled)):
    print(preds_test_unsampled[i].shape)

(256, 256)
(519, 253)
(256, 256)
(256, 256)
(256, 256)
(512, 680)
(512, 640)
(512, 680)
(512, 640)
(256, 256)
(519, 162)
(512, 640)
(524, 348)
(256, 256)
(512, 680)
(256, 256)
(256, 256)
(519, 253)
(256, 256)
(524, 348)
(520, 348)
(260, 347)
(512, 680)
(256, 256)
(512, 680)
(256, 256)
(390, 239)
(512, 640)
(256, 256)
(256, 256)
(260, 347)
(256, 256)
(520, 348)
(512, 640)
(256, 256)
(256, 256)
(260, 347)
(520, 348)
(512, 640)
(256, 256)
(512, 640)
(520, 696)
(256, 256)
(512, 640)
(512, 680)
(519, 253)
(519, 161)
(256, 256)
(520, 696)
(524, 348)
(520, 696)
(524, 348)
(519, 161)
(519, 162)
(520, 348)
(256, 256)
(512, 680)
(256, 256)
(260, 347)
(256, 256)
(519, 253)
(520, 696)
(512, 680)
(256, 256)
(256, 256)
